In [34]:
from trainlib.ModelCollectionConfigFileHandler import ModelCollectionConfigFileHandler
from trainlib.generator import Generator
from trainlib.config import Config
from keras import optimizers

In [ ]:
# this computes low-level performance metrics for a model collection, i.e. the mean-quare error 
# computed on the validation dataset for each discriminant. Since the validation datasets will be held constant,
# this is an easy way to directly compare different models

In [4]:
setting_dir = "/data_CMS/cms/wind/180415_Adam_largebatch/number_layers_2.0_number_neurons_128.0/"
training_dir = "/data_CMS/cms/wind/180415_Adam_largebatch/number_layers_2.0_number_neurons_128.0/training/"

In [5]:
# first, need to read in the trained ModelCollection:
confhandler = ModelCollectionConfigFileHandler()
confhandler.load_configuration(setting_dir + "settings.conf")

attempting to load configuration file from /data_CMS/cms/wind/180415_Adam_largebatch/number_layers_2.0_number_neurons_128.0/settings.conf


In [6]:
mcolls = confhandler.GetModelCollection(weightpath = training_dir)

got the following list of hyperparams: {u'number_layers': 2.0, u'number_neurons': 128.0}
building network with 24 inputs
FlexiblePCAPreprocessor set up for 24 inputs
got the following list of hyperparams: {u'number_layers': 2.0, u'number_neurons': 128.0}
building network with 6 inputs
FlexiblePCAPreprocessor set up for 6 inputs
got the following list of hyperparams: {u'number_layers': 2.0, u'number_neurons': 128.0}
building network with 15 inputs
FlexiblePCAPreprocessor set up for 15 inputs
found the following models belonging to this collection:
D_VBF_ggH_2j_ML
D_VBF_ggH_0j_ML
D_VBF_ggH_1j_ML
now attempting to load model from file /data_CMS/cms/wind/180415_Adam_largebatch/number_layers_2.0_number_neurons_128.0/training/D_VBF_ggH_ML/D_VBF_ggH_2j_ML/checkpoint.hdf5
read weights from /data_CMS/cms/wind/180415_Adam_largebatch/number_layers_2.0_number_neurons_128.0/training/D_VBF_ggH_ML/D_VBF_ggH_2j_ML/checkpoint.hdf5
now attempting to load column settings from file /data_CMS/cms/wind/1804

2018-04-17 18:12:02.718131: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-04-17 18:12:02.718157: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-04-17 18:12:02.718164: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.


In [ ]:
# for the evaluation, need to proceed in the same way as for training, but evaluate the models on the validation
# data instead of training them on the training data

In [10]:
mcoll = mcolls[0]

In [11]:
models, preprocessors, settings = mcoll.get_models(), mcoll.get_preprocessors(), mcoll.get_settings()

In [28]:
    def _compute_class_weights(gen, preprocessor, MC_weighting = False):
        # determine the actual size of the available dataset and adjust the sample weights correspondingly
        H1_data = gen.H1_collection.get_data(Config.branches, 0.0, 1.0)
        H0_data = gen.H0_collection.get_data(Config.branches, 0.0, 1.0)
        H1_length = len(preprocessor.process(H1_data).values()[0])
        H1_indices = preprocessor.get_last_indices()
        H0_length = len(preprocessor.process(H0_data).values()[0])
        H0_indices = preprocessor.get_last_indices()

        print "H1_length = " + str(H1_length)
        print "H0_length = " + str(H0_length)

        # if per-sample weighting is enabled, also set up the normalization of the event weights
        if MC_weighting:
            H1_weight_sum = np.sum(np.maximum(np.array(H1_data["training_weight"][H1_indices]), 0.0))
            H0_weight_sum = np.sum(np.maximum(np.array(H0_data["training_weight"][H0_indices]), 0.0))
        
            H1_class_weight = float(H0_length) / H1_weight_sum
            H0_class_weight = float(H1_length) / H0_weight_sum
        else:
            # H1_class_weight = 1.0
            # H0_class_weight = float(H1_length) / float(H0_length)
            H1_class_weight = 1.0 + float(H0_length) / float(H1_length)
            H0_class_weight = 1.0 + float(H1_length) / float(H0_length)

        return H1_class_weight, H0_class_weight

In [ ]:
for cur_model, cur_preprocessor, cur_settings in zip(models, preprocessors, settings):
    val_gen = Generator(mcoll.H1_stream, mcoll.H0_stream, Config.branches, preprocessor = cur_preprocessor, chunks = 1, MC_weighting = False)
    val_gen.setup_validation_data()
    val_H1_classweight, val_H0_classweight = _compute_class_weights(val_gen, cur_preprocessor, False)
    print val_H1_classweight
    print val_H0_classweight
    val_gen.set_H1_weight(val_H1_classweight)
    val_gen.set_H0_weight(val_H0_classweight)
    val_gen.set_minimum_length(0)
    cur_model.get_keras_model().compile(optimizer = optimizers.Adam(), loss = "mean_squared_error", metrics = ["binary_accuracy"])
    res = cur_model.get_keras_model().evaluate_generator(val_gen.preprocessed_generator(), steps = 1)
    print res

skimming /data_CMS/cms/wind/CJLST_NTuples/VBFH125/ZZ4lAnalysis.root
collection set up: 1 files, 62320 entries in total, 15580 of which will be used
skimming /data_CMS/cms/wind/CJLST_NTuples/ggH125/ZZ4lAnalysis.root
collection set up: 1 files, 110483 entries in total, 27621 of which will be used
